In [1]:
import matplotlib.pyplot as plt 
import SimpleITK as sitk 
import os
from misc import Config
from model import *
%matplotlib qt
os.pathsep = "/"

In [2]:
ELASTIX_PATH = os.path.join('../../elastix/elastix.exe')
TRANSFORMIX_PATH = os.path.join('../../elastix/transformix.exe')
FOLDER_RAW = os.path.join('../../Project/Data')
FOLDER_PREPROCESSED = os.path.join('../../Project/DataPreProcessed')
FOLDER_PARAMETERS = os.path.join('../../Project/parameters')
FOLDER_RESULTS = os.path.join('../../Project/results')

In [3]:
config = Config(
    folder_raw = FOLDER_RAW, 
    folder_preprocessed = FOLDER_PREPROCESSED,
    parameter_folder = FOLDER_PARAMETERS,
    parameters = ["Par0001translation.txt", "Par0001rigid.txt", "Par0001affine.txt", "Par0001bspline04.txt"],
    folder_results = FOLDER_RESULTS,
    seed = 0,
    train_size = 12,
)

In [4]:
model = Model(
    config = config,
    elastix_path = ELASTIX_PATH,
    transformix_path = TRANSFORMIX_PATH,
)

In [5]:
model.load()

Loading Data: 100%|██████████| 15/15 [00:02<00:00,  6.73it/s]


In [6]:
model.preprocess_and_save(downscale_factor=2, normalize="none")

Saving Preprocessed Data: 100%|██████████| 15/15 [00:03<00:00,  4.75it/s]


In [7]:
model.run_registration()

Running with parameters ['Par0001translation.txt', 'Par0001rigid.txt', 'Par0001affine.txt', 'Par0001bspline04.txt']


  0%|          | 0/3 [00:00<?, ?it/s]

../../elastix/elastix.exe -f ../../Project/DataPreProcessed\p107\mr_bffe.mhd -m ../../Project/DataPreProcessed\p125\mr_bffe.mhd -p ../../Project/parameters\Par0001translation.txt -p ../../Project/parameters\Par0001rigid.txt -p ../../Project/parameters\Par0001affine.txt -p ../../Project/parameters\Par0001bspline04.txt -out ../../Project/results\23-03-04_11-44-08\p107\p125
../../elastix/elastix.exe -f ../../Project/DataPreProcessed\p107\mr_bffe.mhd -m ../../Project/DataPreProcessed\p135\mr_bffe.mhd -p ../../Project/parameters\Par0001translation.txt -p ../../Project/parameters\Par0001rigid.txt -p ../../Project/parameters\Par0001affine.txt -p ../../Project/parameters\Par0001bspline04.txt -out ../../Project/results\23-03-04_11-44-08\p107\p135


In [ ]:
model.threshold_staple = 0.95
model.calculate_scores()

Running for parameters ['Par0001rigid.txt', 'Par0001affine.txt', 'Par0001bspline04.txt']


  0%|          | 0/3 [00:28<?, ?it/s]


ValueError: cannot use columns parameter with orient='columns'

In [ ]:
model.scores